# TP3 - Find the soda logo

In [5]:
%matplotlib qt
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

## Find the soda logo within the provided images

In [6]:
# Create results directory if it doesn't exist
os.makedirs('results', exist_ok=True)

1. Obtain a logo detection in each image without false positives

In [7]:
# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def load_image(path, max_size=1200):
    """Load image and return RGB, grayscale, and BGR versions."""
    img = cv.imread(path)
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        img = cv.resize(img, None, fx=scale, fy=scale)
    
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return img_rgb, img_gray, img


def load_template(path, max_size=400):
    """Load template as grayscale."""
    template = cv.imread(path, 0)
    h, w = template.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        template = cv.resize(template, None, fx=scale, fy=scale)
    return template


# Note: The create_template_variants function is defined in Cell 5 (DETECTION FUNCTIONS)
# This cell only contains utility functions for loading and visualization


def draw_detections(img_rgb, detections):
    """Draw detection rectangles on image with confidence scores."""
    img_out = img_rgb.copy()
    for det in detections:
        x, y, w, h = det[:4]
        score = det[4] if len(det) > 4 else 1.0
        
        # Color according to confidence level (green high, yellow medium, red low)
        if score >= 0.7:
            color = (0, 255, 0)  # Green
        elif score >= 0.5:
            color = (255, 255, 0)  # Yellow
        else:
            color = (255, 0, 0)  # Red
        
        # Draw rectangle
        cv.rectangle(img_out, (x, y), (x + w, y + h), color, 2)
        
        # Display confidence level
        label = f'{score:.2f}'
        label_size, _ = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        label_y = max(y - 5, label_size[1])
        cv.rectangle(img_out, (x, label_y - label_size[1] - 5), 
                    (x + label_size[0], label_y + 5), color, -1)
        cv.putText(img_out, label, (x, label_y), 
                  cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    return img_out


def plot_results_grid(results, title, save_path=None):
    """Plot results in a grid layout with confidence information."""
    n = len(results)
    n_cols = 3
    n_rows = (n + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten()
    
    for idx, (img_rgb, detections, img_name) in enumerate(results):
        img_result = draw_detections(img_rgb, detections)
        axes[idx].imshow(img_result)
        
        # Display confidence information in the title
        if detections:
            scores = [d[4] for d in detections if len(d) > 4]
            avg_conf = np.mean(scores) if scores else 0.0
            conf_str = f"avg confidence: {avg_conf:.2f}" if len(detections) > 1 else f"confidence: {scores[0]:.2f}"
            axes[idx].set_title(f"{img_name}\n{len(detections)} detection(s), {conf_str}")
        else:
            axes[idx].set_title(f"{img_name}\n0 detections")
        axes[idx].axis('off')
    
    for idx in range(n, len(axes)):
        axes[idx].axis('off')
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=120, bbox_inches='tight')
    plt.show()


def plot_single_result(img_rgb, detections, title, save_path=None):
    """Plot single image result with confidence information."""
    img_result = draw_detections(img_rgb, detections)
    plt.figure(figsize=(12, 10))
    plt.imshow(img_result)
    
    # Display confidence information in the title
    if detections:
        scores = [d[4] for d in detections if len(d) > 4]
        avg_conf = np.mean(scores) if scores else 0.0
        min_conf = np.min(scores) if scores else 0.0
        max_conf = np.max(scores) if scores else 0.0
        title_with_conf = f"{title} | {len(detections)} detections\nConfidence: min={min_conf:.2f}, avg={avg_conf:.2f}, max={max_conf:.2f}"
    else:
        title_with_conf = f"{title} | 0 detections"
    
    plt.title(title_with_conf, fontsize=12)
    plt.axis('off')
    if save_path:
        plt.savefig(save_path, dpi=120, bbox_inches='tight')
    plt.show()

In [ ]:
# =============================================================================
# DETECTION FUNCTIONS
# =============================================================================

def create_template_variants(template):
    """
    Create useful template variants for template matching.
    Preprocessing matches image preprocessing for better alignment.
    """
    # Apply CLAHE first (matching image preprocessing)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_template = clahe.apply(template)
    
    # Smooth variant (with CLAHE applied)
    smooth = cv.GaussianBlur(clahe_template, (3, 3), 0)
    
    # Inverted variant (with CLAHE applied)
    inverted = 255 - clahe_template
    
    # Equalized histogram variant (alternative preprocessing)
    equalized = cv.equalizeHist(template)
    
    return {
        'original': clahe_template,  # Use CLAHE as base to match image preprocessing
        'clahe': clahe_template,
        'smooth': smooth,
        'inverted': inverted,
        'equalized': equalized,
    }


def preprocess_image(img_gray):
    """
    Preprocess image before template matching.
    Uses CLAHE to match template preprocessing for better alignment.
    """
    # Apply CLAHE for contrast normalization (matching template preprocessing)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img_gray)
    # Light Gaussian blur to reduce noise while preserving edges
    img = cv.GaussianBlur(img, (3, 3), 0)
    return img


def compute_iou(box1, box2):
    """Compute IoU between two boxes (x, y, w, h)."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_w = max(0, xi2 - xi1)
    inter_h = max(0, yi2 - yi1)
    inter_area = inter_w * inter_h
    
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - inter_area
    
    return inter_area / union_area if union_area > 0 else 0


def nms_global(detections, iou_threshold=0.3):
    """
    Apply Non-Maximum Suppression globally across all detections.
    detections: list of (x, y, w, h, score)
    """
    if not detections:
        return []
    
    detections = sorted(detections, key=lambda d: d[4], reverse=True)
    
    keep = []
    while detections:
        best = detections.pop(0)
        keep.append(best)
        detections = [d for d in detections if compute_iou(best[:4], d[:4]) < iou_threshold]
    
    return keep


def find_by_template(img_gray, template_variants, single_detection=True, min_threshold=0.30):
    """
    Multi-scale template matching following the class code pattern.
    - 30 scales between 0.2 and 1.6
    - Adaptive threshold: mean + multiplier*std, min threshold according to mode
    - Global NMS to filter overlaps
    - Aspect ratio filtering: 0.5 < w/h < 3.0
    
    For multiple detections, uses lower threshold (following 01.Templates.ipynb Cell 6).
    Improved thresholds based on actual detection confidence levels observed.
    """
    h, w = img_gray.shape
    img_processed = preprocess_image(img_gray)
    
    # Adjust threshold according to detection mode
    # Based on observed confidence levels (0.35-0.70 range), thresholds adjusted
    if single_detection:
        base_threshold = 0.25  # Lowered from 0.30 to catch detections with confidence ~0.35-0.45
        adaptive_multiplier = 1.5  # Reduced from 2.5 to be less strict (allows more detections)
    else:
        base_threshold = 0.35  # Lowered from 0.60 for multiple detections (matches observed confidence range)
        adaptive_multiplier = 1.2  # More permissive for multiple detections
    
    scales = np.linspace(0.2, 1.6, 30)
    all_detections = []
    
    for scale in scales:
        for name, tmpl in template_variants.items():
            th, tw = tmpl.shape
            new_w = int(tw * scale)
            new_h = int(th * scale)
            
            if new_w > w or new_h > h or new_w < 20 or new_h < 10:
                continue
            
            scaled_tmpl = cv.resize(tmpl, (new_w, new_h))
            # Use TM_CCOEFF_NORMED as in 01.Templates.ipynb Cell 6
            res = cv.matchTemplate(img_processed, scaled_tmpl, cv.TM_CCOEFF_NORMED)
            
            # Adaptive threshold with reduced multiplier for better sensitivity
            mean_val = np.mean(res)
            std_val = np.std(res)
            threshold = max(mean_val + adaptive_multiplier * std_val, base_threshold)
            
            # np.where() as in 01.Templates.ipynb Cell 6
            locations = np.where(res >= threshold)
            
            for pt in zip(*locations[::-1]):
                x, y = pt
                score = res[y, x]
                
                aspect = new_w / new_h if new_h > 0 else 0
                if not (0.5 < aspect < 3.0):
                    continue
                
                all_detections.append((x, y, new_w, new_h, score))
    
    if not all_detections:
        return (None, None, 0) if single_detection else []
    
    # Apply NMS to filter overlaps (improvement over basic 01.Templates.ipynb)
    final_detections = nms_global(all_detections, iou_threshold=0.3)
    
    if single_detection:
        best = final_detections[0]
        return (best[0], best[1], best[2], best[3]), 'template', best[4]
    
    return final_detections


def detect_logo(img_bgr, img_gray, template_variants, single_detection=True):
    """
    Detection pipeline using only template matching (aligned with 01.Templates.ipynb).
    - Single detection: Template matching with strict threshold
    - Multi detection: Template matching with more permissive threshold (like 01.Templates.ipynb Cell 6)
    """
    if single_detection:
        bbox, method, score = find_by_template(img_gray, template_variants, single_detection=True)
        if bbox:
            return bbox, method, score
        return None, None, 0
    else:
        # Multiple detections using template matching (like 01.Templates.ipynb Cell 6)
        # Lower threshold for multiple detections to catch all instances
        detections = find_by_template(img_gray, template_variants, single_detection=False, min_threshold=0.35)
        return detections

In [ ]:
# =============================================================================
# TEMPLATE VARIANTS VISUALIZATION (4 useful transformations)
# =============================================================================

TEMPLATE_PATH = 'template/pattern.png'
template = load_template(TEMPLATE_PATH)
variants = create_template_variants(template)

fig, axes = plt.subplots(1, 5, figsize=(18, 4))

for ax, (name, img) in zip(axes, variants.items()):
    ax.imshow(img, cmap='gray')
    ax.set_title(name)
    ax.axis('off')

fig.suptitle(f'Template Variants ({len(variants)} transformations)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('results/template_variants.png', dpi=120, bbox_inches='tight')
plt.show()

In [10]:
# =============================================================================
# ASSIGNMENT 1: Single Detection per Image
# =============================================================================

IMAGES_DIR = 'images/'
image_files = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(('.png', '.jpg'))])

results = []
for img_name in image_files:
    img_path = os.path.join(IMAGES_DIR, img_name)
    img_rgb, img_gray, img_bgr = load_image(img_path)
    
    bbox, method, match_val = detect_logo(img_bgr, img_gray, variants, single_detection=True)
    
    if bbox:
        x, y, w, h = bbox
        # Save with real confidence score
        detections = [(x, y, w, h, match_val)]
        print(f"{img_name}: {method} (confidence={match_val:.3f})")
    else:
        detections = []
        print(f"{img_name}: NOT DETECTED")
    
    results.append((img_rgb, detections, img_name))

plot_results_grid(results, "ASSIGNMENT 1: Single Detection per Image", 'results/Figure_1.png')

COCA-COLA-LOGO.jpg: template (confidence=0.415)
coca_logo_1.png: template (confidence=0.358)
coca_logo_2.png: template (confidence=0.395)
coca_multi.png: template (confidence=0.457)
coca_retro_1.png: template (confidence=0.631)
coca_retro_2.png: template (confidence=0.699)
logo_1.png: template (confidence=0.435)


2. Propose and validate an algorithm for multiple detections in the image coca_multi.png using the same template from item 1

In [11]:
# =============================================================================
# ASSIGNMENT 2: Multiple Detections on coca_multi.png
# Using template matching with the same template from Item 1 (like 01.Templates.ipynb Cell 6)
# =============================================================================

img_rgb, img_gray, img_bgr = load_image('images/coca_multi.png')
detections = detect_logo(img_bgr, img_gray, variants, single_detection=False)

print(f"Detected {len(detections)} logo instances in coca_multi.png")
for i, det in enumerate(detections):
    x, y, w, h, score = det
    print(f"  Detection {i+1}: position=({x},{y}), size=({w}x{h}), confidence={score:.3f}")

plot_single_result(img_rgb, detections, "ASSIGNMENT 2: coca_multi.png (Template Matching)", 'results/Figure_2.png')

Detected 0 logo instances in coca_multi.png


3. Generalize the algorithm from item 2 for all images

In [12]:
# =============================================================================
# ASSIGNMENT 3: Generalized Algorithm for All Images
# Generalizes the template matching algorithm from Item 2 for all images
# =============================================================================

results_single = []
results_multi = []

for img_name in image_files:
    img_rgb, img_gray, img_bgr = load_image(os.path.join(IMAGES_DIR, img_name))
    
    # Single detection: template matching with strict threshold
    bbox, method, score = detect_logo(img_bgr, img_gray, variants, single_detection=True)
    det_single = [(bbox[0], bbox[1], bbox[2], bbox[3], score)] if bbox else []
    results_single.append((img_rgb, det_single, img_name))
    
    # Multiple detection: template matching with more permissive threshold (Item 2 algorithm)
    det_multi = detect_logo(img_bgr, img_gray, variants, single_detection=False)
    results_multi.append((img_rgb, det_multi, img_name))
    
    print(f"{img_name}: {len(det_single)} single detection, {len(det_multi)} multiple detections")

plot_results_grid(results_single, "ASSIGNMENT 3: Single Detection (Template Matching)", 'results/Figure_3a_single.png')
plot_results_grid(results_multi, "ASSIGNMENT 3: Multiple Detection (Template Matching)", 'results/Figure_3b_multi.png')

COCA-COLA-LOGO.jpg: 1 single detection, 0 multiple detections
coca_logo_1.png: 1 single detection, 0 multiple detections
coca_logo_2.png: 1 single detection, 0 multiple detections
coca_multi.png: 1 single detection, 0 multiple detections
coca_retro_1.png: 1 single detection, 1 multiple detections
coca_retro_2.png: 1 single detection, 1 multiple detections
logo_1.png: 1 single detection, 0 multiple detections
